<h1 id="tocheading">Table of Contents</h1>
<div id="toc"></div>
<script src="https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js"></script>

type1の店(tokubai.co.jpのリンクを踏むとチラシの画像が表示されるような店)

type2の店(tokubai.co.jpのリンクを踏むと具体的な特売商品が文字列で表示されるような店)

# 全体の流れ
- 1.リスト`links1`にチラシを取得する店の階層1リンクを格納(例:https://tokubai.co.jp/%E3%83%A9%E3%82%A4%E3%83%95/9470)
- 2.店ごとに取得する条件が異なるため、`getlinks12(link1)`という関数を作成。この関数は`links1`に含まれる適切なチラシの階層2リンク(string)のlist(`links2raw`)を返す
- 3.`links2raw`には最新ではないチラシも含まれるため、関数`get_new_urls(links2raw)`を作成。`./urls.txt`を利用。返り値`links2`には新しいチラシの階層2リンク
- [4.`get_urls()`で階層2のリンクを階層3のリンク`links3`に変換](#4.`get_urls()`で階層2のリンクを階層3のリンクに変換 )

- ([5.`DLjpg()`で内部にダウンロード](#5.`DLjpg()`で内部にダウンロード))

In [ ]:
#関数設定した後に実行する部分
links1 = ["url_life_kitashirakawa","url_coremo","url_life_higasi1jou","url..."]
links2raw = link1_to_link2(links1)
links2 = get_new_urls=(links2raw)
links3 = get_urls(links2)
DLjpg(links3)

## 事前準備

In [1]:
import requests
from bs4 import BeautifulSoup
import calendar
#ユーザエージェント変更
ua = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36"
headers = {'User-Agent': ua}

## 1.リスト`links1`にチラシを取得する店の階層1リンクを格納
一旦ライフとグレースたなかだけで

In [8]:
links1 = []
links1.append("https://tokubai.co.jp/%E3%83%A9%E3%82%A4%E3%83%95/9470") #ライフ
links1.append("https://tokubai.co.jp/%E3%82%B0%E3%83%AC%E3%83%BC%E3%82%B9%E3%81%9F%E3%81%AA%E3%81%8B/12265") #グレースたなか

## 2.店ごとに取得する条件が異なるため、`link1_to_link2(link1)`という関数を作成。この関数は、リスト`links1`に含まれる適切なチラシの階層2リンク(string)のlist(`links2raw`)を返す

ライフとグレースたなかだけなのは同様


In [3]:
def link1_to_link2(links1):
    links2raw = [] #最終的には取得日に乗っているチラシのリンク(link2)一覧が格納される
    for link1 in links1:
        if "%E3%83%A9%E3%82%A4%E3%83%95" in link1: #ライフの場合
            r = requests.get(link1,headers=headers)
            soup = BeautifulSoup(r.text,"html.parser")
            elements = soup.find_all(class_="image_element")
            for element in elements:
                link = element.get("href")
                l = "https://tokubai.co.jp"
                date = element.find(class_="description").string
                if date.count("月") == 1: #チラシ掲載期間が月を跨がない場合 例：/n2023年4月2〜4日まで/n -> 2023,4,2,4
                    date = date[1:-1].replace("月",",").replace("〜",",").replace("日まで",",").replace("年",",")
                    dates = date.split(",")
                    period = int(dates[3]) - int(dates[2])
                
                else: #チラシ掲載期間が月を跨ぐ場合　例：/n2023年4月29日〜5月2日まで/n  -> 2023,4,29,5,2
                    date = date[1:-1].replace("月",",").replace("日〜",",").replace("日まで",",").replace("年",",")
                    dates = date.split(",")
                    period = int(dates[4]) - int(dates[2]) + calendar.monthrange(2023, int(dates[1])+1)[1]
                if period < 6: #ライフでは一般的に掲載期間が5日以下のものが取得したいチラシなので
                    links2raw.append(l+link)

        elif "%E3%82%B0%E3%83%AC%E3%83%BC%E3%82%B9%E3%81%9F%E3%81%AA%E3%81%8B" in link1: #グレースたなかの場合
            r = requests.get(link1,headers=headers)
            soup = BeautifulSoup(r.text,"html.parser")
            elements = soup.find_all(class_="image_element")
            for element in elements:
                link = element.get("href")
                l = "https://tokubai.co.jp"
                date = element.find(class_="description").string
                links2raw.append(l+link)

    return links2raw

In [14]:
link1_to_link2(links1)

['https://tokubai.co.jp/%E3%83%A9%E3%82%A4%E3%83%95/9470/leaflets/41521798',
 'https://tokubai.co.jp/%E3%83%A9%E3%82%A4%E3%83%95/9470/leaflets/41521824',
 'https://tokubai.co.jp/%E3%82%B0%E3%83%AC%E3%83%BC%E3%82%B9%E3%81%9F%E3%81%AA%E3%81%8B/12265/leaflets/41550454',
 'https://tokubai.co.jp/%E3%82%B0%E3%83%AC%E3%83%BC%E3%82%B9%E3%81%9F%E3%81%AA%E3%81%8B/12265/leaflets/41550455']

## 3.`links2raw`には最新ではないチラシも含まれるため、関数`get_new_urls(links2raw)`を作成。`./urls.txt`を利用。返り値`links2`には新しいチラシの階層2リンク

In [4]:
def get_new_urls(links2raw):
    new_urls_list = []

    with open('urls.txt',"r") as f: #urls.txtの中には昨日使用したlink2リンクの一覧がある
        lines = f.readlines()
        lines = [line.replace("¥n","").replace("\n","") for line in lines] #改行文字削除 怖かったので\と¥を書いた

    for link2raw in links2raw:
        if link2raw in lines: #昨日使用したリンクは無視
            #print(f"link2raw {link2raw} is in links")
            continue
        else: #昨日なかったリンクは欲しいのでnew_urls_listに格納
            new_urls_list.append(link2raw)
            
    urls_new = map(lambda x: x + "\n", links2raw)
    #links2rawの内容をurls.txtに保存
    with open('urls.txt', 'w', encoding='utf-8') as f:
        f.writelines(urls_new)

    return new_urls_list

## 4.`get_urls()`で階層2のリンクを階層3のリンクに変換

In [5]:
#まとめ： 関数内のurlにあるチラシ.jpgのリンクを出力する`get_url()`
def get_url(urls):
  new_urls = []
  for url in urls:
    r = requests.get(url,headers=headers)
    soup = BeautifulSoup(r.text,"html.parser")
    new_urls.append(soup.find('img',class_='leaflet').get('src')) 
  return new_urls

In [11]:
#関数設定した後に実行する部分
#links1 = ["url_life_kitashirakawa","url_coremo","url_life_higasi1jou","url..."]
links2raw = link1_to_link2(links1)
links2 = get_new_urls=(links2raw)
links3 = get_url(links2)
#DLjpg(links3)

In [12]:
links3

['https://image.tokubai.co.jp/images/bargain_office_leaflets/o=true/5887777.jpg?1682765571',
 'https://image.tokubai.co.jp/images/bargain_office_leaflets/o=true/5887778.jpg?1682765611',
 'https://image.tokubai.co.jp/images/bargain_leaflets/o=true/41717293.jpg?1683351879',
 'https://image.tokubai.co.jp/images/bargain_leaflets/o=true/41717294.jpg?1683351901']

## 5.`DLjpg()`で内部にダウンロード

In [ ]:
def DLjpg(links3):
    for link in links3:
        